In [1]:
import numpy as np
import tiktoken
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pickle
import re
import random
import string
import copy

# Pickle Functions

In [122]:
def save_pkl(file_path, data):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

In [123]:
def get_pkl(file_path):
    with open(file_path, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

# Get data from txt

In [76]:
dataset_pos_path = 'dataset/dataset.txt'
dataset2_path = 'dataset/dataset2.txt'
dataset_neg_path = 'dataset/negative.txt'

In [77]:
with open(dataset_pos_path, 'r') as file:
    pos_sentences = file.read().split('\n')

In [78]:
with open(dataset_neg_path, 'r') as file:
    neg_sentences = file.read().split('\n')

In [79]:
with open(dataset2_path, 'r') as file:
    pos_text = file.read().split('\n')

In [6]:
len(pos_text)

21

# delete "Mount "

In [80]:
def delete_mount(sentence, deletion_probability):
    if 'Mount' in sentence and random.random() < deletion_probability:
        return sentence.replace('Mount ', '')
    return sentence

# Delete 'Mount' with the defined probability in each label
mod_pos_sentences = [delete_mount(sentence, 0.9) for sentence in pos_sentences]

In [81]:
mod_pos_text = [delete_mount(text, 0.8) for text in pos_text]

In [92]:
mod_pos_text

["In a remote corner of the Andes stands a formidable peak, its majesty concealed by the shroud of mystery that envelops itâ€”the [Cerro Torre]. This breathtaking monolith, nestled within the Southern Patagonian Ice Field, presents a daunting challenge to even the most seasoned climbers. With its sheer granite walls and constantly shifting weather patterns, the [Cerro Torre] commands both respect and awe from adventurers who dare to approach its slopes. Legends whisper tales of the mountain's elusive summit, where the forces of nature seem to converge in a mesmerizing dance of ice and wind. The [Cerro Torre] stands not only as a pinnacle of geological marvel but also as a testament to the unwavering spirit of exploration that draws mountaineers to its lofty embrace.",
 'In the heart of the Rwenzori Mountains, obscured by mist and lush vegetation, lies the enigmatic [Stanley]. Rising above the African plains, this snow-capped wonder casts an alluring spell upon those who venture into it

In [83]:
mod_pos_sentences[0]

'Majestic [Everest] stood tall against the dawn sky, a silent guardian of the Himalayan wilderness.'

# Mix sentences

In [84]:
def splitncombine_values(data, n_parts):
    output = list()
    temp = list()
    for i in range(1, n_parts+1):
        temp.append(data[int(((i-1)/n_parts) * len(data)):int(((i)/n_parts) * len(data))])
    return output       

In [85]:
def combine_values_3list(list1, list2, list3):
    output_list = []
    for elem1, elem2, elem3 in zip(list1, list2, list3):
        output_list.append(elem1 + " " + elem2 + " " + elem3)
    return output_list 

In [86]:
def combine_list(data):
    output = list()
    min_len = min([len(i) for i in data])
    for i in range(min_len):
        text = ""
        for j in data:
            text += j[i] + " "
        output.append(text[:-1])

    return output

In [87]:
def combine_values_2list(pos_list, neg_list):
    output_list = []
    for pos, neg in zip(pos_list, neg_list):
        output_list.append(pos + " " + neg)
    return output_list 

In [88]:
def split_list(data, n_parts):
    temp = list()
    for i in range(1, n_parts+1):
        temp.append(data[int(((i-1)/n_parts) * len(data)):int(((i)/n_parts) * len(data))])

    return temp

In [89]:
def Mix_sentences(original_positive, original_negative, list_of_persent):
    positive = copy.deepcopy(original_positive)
    negative = copy.deepcopy(original_negative)

    random.shuffle(positive)
    random.shuffle(negative)
    Pl = len(positive)
    Nl = len(negative)
    
    P = positive[:int(list_of_persent[0] * Pl)]
    N = negative[:int(list_of_persent[0] * Nl)]
    P2 = positive[len(P) : len(P) + int(list_of_persent[1] * Pl)]
    N2 = negative[len(N) : len(N) + int(list_of_persent[1] * Nl)]
    P3 = positive[len(P2):]
    N3 = negative[len(N2):]
    
    Nto2 = N2[:int(len(N2) * 0.8)]
    Pto2 = P2[:len(Nto2)]
    
    PN = combine_values_2list(Pto2[:int(len(Pto2)*0.5)], Nto2[:int(len(Nto2)*0.5)])
    NP = combine_values_2list(Nto2[int(len(Nto2)*0.5):], Pto2[int(len(Pto2)*0.5):])
    PP = combine_list(split_list(P2[len(Nto2):], 2))
    NN = combine_list(split_list(N2[len(Nto2):], 2))

    Nto3 = N3[:int(len(N3) * 0.8)]
    Pto3 = P3[:len(Nto3)]

    Ns3 = split_list(Nto3, 9)
    Ps3 = split_list(Pto3, 9)
    PPN = combine_values_3list(Ps3[0], Ps3[1], Ns3[0])
    NPP = combine_values_3list(Ns3[1], Ps3[2], Ps3[3])
    NPN = combine_values_3list(Ns3[2], Ps3[4], Ns3[3])
    PNP = combine_values_3list(Ps3[5], Ns3[4], Ps3[6])
    NNP = combine_values_3list(Ns3[5], Ns3[6], Ps3[7])
    PNN = combine_values_3list(Ps3[8], Ns3[7], Ns3[8])
    PPP = combine_list(split_list(P3[len(Nto3):], 3))
    NNN = combine_list(split_list(N3[len(Nto3):], 3))

    output = list()
    for i in [P, N, PN, NP, PP, NN, PPN, NPP, NPN, PNP, NNP, PNN, PPP, NNN]:
        output.extend(i)
    return output

In [90]:
dataset = Mix_sentences(mod_pos_sentences, neg_sentences, [0.1, 0.25])

In [91]:
len(dataset)

229

# data union

In [93]:
dataset.extend(mod_pos_text)
len(dataset)

250

# Lowercase

In [99]:
dataset_l = [sentence.lower() for sentence in dataset]

In [102]:
dataset_l

['within the bustling marketplace, traders exchanged goods sourced from villages near [mount mckinley].',
 'the air at [aconcagua]â€™s peak whispers tales of endurance and resilience.',
 'the breathtaking view from the summit of [aconcagua], the highest peak in south america, is an unforgettable experience.',
 'the mystical aura surrounding [machu picchu] in peru evokes a sense of wonder, steeped in ancient history and architectural brilliance.',
 'the architect envisioned sustainable designs inspired by the symmetry of [mount damavand].',
 '[denali], a realm of icy splendor, challenges adventurers to surpass their limits.',
 'from the balcony, one could glimpse the snow-capped peak of [rainier].',
 'deep within the himalayas lies [annapurna], a symbol of both grace and daunting challenge for mountaineers.',
 'the dance of light and shadow painted a mystical aura around [whitney].',
 'the rockies boast numerous peaks, each more impressive than the last, from the iconic [pikes peak] to 

# Exstract labels from text

In [96]:
def get_labels(sentences):
    list_of_labels = list()
    for sent in sentences:
        extracted_values = re.findall(r'\[(.*?)\]', sent)
        label = ', '.join(extracted_values)
        list_of_labels.append(label)
    return list_of_labels

In [100]:
labels = get_labels(dataset_l)

In [101]:
labels

['mount mckinley',
 'aconcagua',
 'aconcagua',
 'machu picchu',
 'mount damavand',
 'denali',
 'rainier',
 'annapurna',
 'whitney',
 'pikes peak, mount rainier',
 'gissar range',
 'blackburn',
 'k2',
 'kosciuszko',
 'aconcagua',
 'saint helena',
 'cook',
 'mount mckinley',
 'k2',
 'baker',
 'everest',
 'olympus',
 'kilimanjaro',
 'marcus baker',
 'huascarã¡n',
 'bogd khan uul',
 'mount vesuvius',
 'elbrus',
 'mount kilimanjaro',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'kebnekaise',
 'fuji',
 'everest',
 'nanga parbat',
 'kinabalu',
 'matterhorn',
 'denali',
 'ararat',
 'aconcagua',
 'isto',
 'gerlachovsky stit',
 'k2',
 'kangchenjunga',
 'annapurna',
 'apo',
 'matterhorn',
 'mount ararat',
 'trolltunga',
 'cook',
 'annapurna',
 'kenya',
 'mount hood',
 'kilimanjaro',
 'mont blanc',
 'blackburn',
 'elbrus',
 'puncak jaya',
 'annapurna',
 'elbrus',
 'mont blanc',
 'tre cime di lavaredo, marmolada',
 'olympus',
 'mitchell',
 'wadding

# punctuation

In [103]:
def remove_punctuation_except_period_apostrophe(text):
    punctuation_to_keep = ".'!?"
    return ''.join(char for char in text if char not in string.punctuation or char in punctuation_to_keep)

In [104]:
def remove_hyperlinks_special_symbol(text):
    # Remove hyperlinks
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove special symbols except for letters, numbers, spaces, periods, and apostrophes
    text = re.sub(r'[^\w\s.\'"]', '', text)
    return text

In [105]:
def replace_punctuation(text):
    return text.replace('!', '.').replace('?', '.')

In [106]:
cleaned_list = [remove_punctuation_except_period_apostrophe(s) for s in dataset_l]
cleaned_list = [remove_hyperlinks_special_symbol(sentence) for sentence in cleaned_list]
cleaned_list = [replace_punctuation(sentence) for sentence in cleaned_list]

In [112]:
cleaned_list

['within the bustling marketplace traders exchanged goods sourced from villages near mount mckinley.',
 'the air at aconcaguaâs peak whispers tales of endurance and resilience.',
 'the breathtaking view from the summit of aconcagua the highest peak in south america is an unforgettable experience.',
 'the mystical aura surrounding machu picchu in peru evokes a sense of wonder steeped in ancient history and architectural brilliance.',
 'the architect envisioned sustainable designs inspired by the symmetry of mount damavand.',
 'denali a realm of icy splendor challenges adventurers to surpass their limits.',
 'from the balcony one could glimpse the snowcapped peak of rainier.',
 'deep within the himalayas lies annapurna a symbol of both grace and daunting challenge for mountaineers.',
 'the dance of light and shadow painted a mystical aura around whitney.',
 'the rockies boast numerous peaks each more impressive than the last from the iconic pikes peak to the imposing presence of mount ra

# Stop words

In [108]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\da4nik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\da4nik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [109]:
stopWords = set(stopwords.words('english'))
print(stopWords)

{'him', 'once', 'to', "you'll", 'not', 'ours', 'our', "aren't", 'she', 'between', 'll', 'ourselves', 'at', 'nor', 'hadn', 'myself', 'it', 'for', 'does', "wasn't", 'because', 'here', 'all', "that'll", "mightn't", 'before', 'in', 'whom', 'he', 'what', "mustn't", 'themselves', 'doesn', 'will', "you've", "won't", 'is', 'won', 'yourselves', 'out', 's', 'again', 'most', "didn't", "you're", 'above', 'which', 'other', 'than', 'too', 'ain', "needn't", "you'd", 'we', 'shouldn', 'having', 'during', 'as', 'm', 'her', 'himself', 'were', 'my', 'herself', 'very', 'them', "she's", 'me', 'those', 'don', "shan't", 'a', 'now', 'i', 'through', 'any', 'wasn', 'same', 'into', 'been', 'hasn', "wouldn't", 'weren', 'his', 'until', 'over', 'only', 'o', 'own', 'mustn', 'few', 'itself', 'yourself', 'did', 'why', 't', 'are', 'have', 'how', "haven't", "weren't", 'with', 'of', 'off', 'you', 'there', 'some', 'and', 'doing', 'each', 'they', 'about', "shouldn't", 'haven', "hadn't", 'more', 'was', 'yours', 'didn', 'its'

In [110]:
def del_stop_word(sentences, stop_words):
    result = list()
    for i in sentences:
        words = nltk.word_tokenize(i)
        filtered_sentence = [word for word in words if word not in stop_words]
        output_sentence = ' '.join(filtered_sentence)
        result.append(output_sentence)
    return result

In [116]:
def replace_space(text):
    return text.replace(" '", "'").replace(" .", '.')

In [113]:
stopl_dataset = del_stop_word(cleaned_list, stopWords)

In [117]:
stopl_dataset = [replace_space(sentence) for sentence in stopl_dataset]

In [118]:
stopl_dataset

['within bustling marketplace traders exchanged goods sourced villages near mount mckinley.',
 'air aconcaguaâs peak whispers tales endurance resilience.',
 'breathtaking view summit aconcagua highest peak south america unforgettable experience.',
 'mystical aura surrounding machu picchu peru evokes sense wonder steeped ancient history architectural brilliance.',
 'architect envisioned sustainable designs inspired symmetry mount damavand.',
 'denali realm icy splendor challenges adventurers surpass limits.',
 'balcony one could glimpse snowcapped peak rainier.',
 'deep within himalayas lies annapurna symbol grace daunting challenge mountaineers.',
 'dance light shadow painted mystical aura around whitney.',
 'rockies boast numerous peaks impressive last iconic pikes peak imposing presence mount rainier.',
 'along silk road ancient city samarkand framed distant silhouette gissar range.',
 'amidst untamed wilderness rumors spoke ancient artifact near blackburn.',
 'storm serene calm sett

# Save dataset

In [120]:
labels

['mount mckinley',
 'aconcagua',
 'aconcagua',
 'machu picchu',
 'mount damavand',
 'denali',
 'rainier',
 'annapurna',
 'whitney',
 'pikes peak, mount rainier',
 'gissar range',
 'blackburn',
 'k2',
 'kosciuszko',
 'aconcagua',
 'saint helena',
 'cook',
 'mount mckinley',
 'k2',
 'baker',
 'everest',
 'olympus',
 'kilimanjaro',
 'marcus baker',
 'huascarã¡n',
 'bogd khan uul',
 'mount vesuvius',
 'elbrus',
 'mount kilimanjaro',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'kebnekaise',
 'fuji',
 'everest',
 'nanga parbat',
 'kinabalu',
 'matterhorn',
 'denali',
 'ararat',
 'aconcagua',
 'isto',
 'gerlachovsky stit',
 'k2',
 'kangchenjunga',
 'annapurna',
 'apo',
 'matterhorn',
 'mount ararat',
 'trolltunga',
 'cook',
 'annapurna',
 'kenya',
 'mount hood',
 'kilimanjaro',
 'mont blanc',
 'blackburn',
 'elbrus',
 'puncak jaya',
 'annapurna',
 'elbrus',
 'mont blanc',
 'tre cime di lavaredo, marmolada',
 'olympus',
 'mitchell',
 'wadding

In [124]:
save_pkl('dataset/y.pkl', labels)
save_pkl('dataset/x.pkl', stopl_dataset)

# Encode tiktoken

In [33]:
enc = tiktoken.get_encoding("cl100k_base")

In [34]:
def tokenization(result, enc):
    output = list()
    for i in result:
        output.append(enc.encode(i))
    return output

In [115]:
enc_sentences = tokenization(mod_pos_text, enc)

In [116]:
len(max(enc_sentences))

138

In [48]:
enc_sentences[0]

[44,
 1662,
 10027,
 510,
 44587,
 478,
 60,
 14980,
 16615,
 2403,
 279,
 39493,
 13180,
 11,
 264,
 21737,
 47132,
 315,
 279,
 76438,
 44957,
 49362,
 13]